In [1]:
pip install -r requirements.txt 

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Using cached ipython-8.12.3-py3-none-any.whl.metadata (5.7 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [11 lines of output]
      
      
      You need Cython to compile Pyjnius.
      
      
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/private/var/folders/9k/rnh189dj1_95w5p98sc88vbh0000gn/T/pip-install-uty990r5/jnius_39a3f826731c46689ceb9ac10de00969/setup.py", line 53, in <module>
          from Cython.Distutils import build_ext
      ModuleNotFoundError: No module named 'Cython'
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata

In [10]:
import subprocess
from pathlib import Path
import pandas as pd
from tqdm import tqdm

# Define paths
dataset_path = Path("data/original_dataset")
classified_cartoon_path = Path("data/original_dataset_cartoon_only/images")
classified_photo_path = Path("data/original_dataset_photo_only/images")

# Function to run the classifier script
def run_classifier(threshold=0.5):
    cmd = [
        "python", "cartoon_classifier.py",
        str(dataset_path),
        "-t", str(threshold)
    ]
    subprocess.run(cmd, check=True)

# Function to check for missing labels
def check_missing_labels(dataset_path, cartoon_path, photo_path):
    # Collect all images and labels
    images = list((dataset_path / "images").glob("**/*.jpg"))
    labels = list((dataset_path / "labels").glob("**/*.txt"))
    
    image_names = {img.stem for img in images}
    label_names = {lbl.stem for lbl in labels}
    
    missing_labels = image_names - label_names
    missing_images = label_names - image_names
    
    if missing_labels:
        print(f"Warning: {len(missing_labels)} images are missing labels.")
    else:
        print("All images have corresponding labels.")
    
    if missing_images:
        print(f"Warning: {len(missing_images)} labels are missing images.")
    else:
        print("All labels have corresponding images.")
    
    return missing_labels, missing_images

# Function to check classified results
def check_classified_results(cartoon_path, photo_path):
    classified_cartoon_images = list(cartoon_path.glob("**/*.jpg"))
    classified_photo_images = list(photo_path.glob("**/*.jpg"))
    
    print(f"Total classified cartoon images: {len(classified_cartoon_images)}")
    print(f"Total classified photo images: {len(classified_photo_images)}")
    
    return classified_cartoon_images, classified_photo_images




In [11]:
# Run the classifier script
run_classifier(threshold=0.3)

Traceback (most recent call last):
  File "/Users/johanbarreiro/Documents/GitHub/DL/cartoon_classifier.py", line 5, in <module>
    from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
ModuleNotFoundError: No module named 'tensorflow'


CalledProcessError: Command '['python', 'cartoon_classifier.py', 'data/original_dataset', '-t', '0.3']' returned non-zero exit status 1.

In [13]:
# Check for missing labels
missing_labels, missing_images = check_missing_labels(dataset_path, classified_cartoon_path, classified_photo_path)

# Check classified results
classified_cartoon_images, classified_photo_images = check_classified_results(classified_cartoon_path, classified_photo_path)

# Display results
missing_labels
missing_images

All labels have corresponding images.
Total classified cartoon images: 12
Total classified photo images: 2398


set()